
# Alura Store Latam — Análisis y Recomendación
Calcula KPIs de las 4 tiendas y recomienda **qué tienda vender** (la menos eficiente), basándose en **ingresos, reseñas y envío**.


In [ ]:

import os, sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (9,5)
plt.rcParams["axes.grid"] = True

ROOT = Path.cwd().parents[1] if (Path.cwd().name == "notebooks") else Path.cwd()
if (ROOT/"src").exists():
    sys.path.append(str(ROOT/"src"))
from metrics import compute_store_metrics, efficiency_score
FIGS = ROOT/"figs"; FIGS.mkdir(exist_ok=True, parents=True)
REPORT = ROOT/"reporte"; REPORT.mkdir(exist_ok=True, parents=True)


In [ ]:

# Carga de datos (URLs oficiales o local en /data con USE_LOCAL=True)
USE_LOCAL = False
base_local = ROOT / "data"
base_url = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam"
files = {"tienda_1": "tienda_1%20.csv", "tienda_2": "tienda_2.csv", "tienda_3": "tienda_3.csv", "tienda_4": "tienda_4.csv"}

def load_df(fname):
    if USE_LOCAL:
        return pd.read_csv(base_local / fname)
    else:
        return pd.read_csv(f"{base_url}/{fname}")

store_dfs = {}
for k, fname in files.items():
    try:
        store_dfs[k] = load_df(fname)
        print("OK:", k, store_dfs[k].shape)
    except Exception as e:
        print("ERROR cargando", k, "->", e)

tienda  = store_dfs.get("tienda_1", pd.DataFrame())
tienda2 = store_dfs.get("tienda_2", pd.DataFrame())
tienda3 = store_dfs.get("tienda_3", pd.DataFrame())
tienda4 = store_dfs.get("tienda_4", pd.DataFrame())

display(tienda.head(3))
display(tienda2.head(3))
display(tienda3.head(3))
display(tienda4.head(3))


In [ ]:

stores = {"Tienda 1": tienda, "Tienda 2": tienda2, "Tienda 3": tienda3, "Tienda 4": tienda4}
kpis, by_category, top_by_store, bottom_by_store = [], {}, {}, {}

for name, df in stores.items():
    m = compute_store_metrics(df, name)
    kpis.append({"store": name,
                 "total_ingresos": m["total_ingresos"],
                 "calificacion_prom": m["calificacion_prom"],
                 "envio_promedio": m["envio_promedio"],
                 "n_filas": len(m["df"])})
    by_category[name] = m["categoria_counts"]
    top_by_store[name] = m["top_prod"]
    bottom_by_store[name] = m["bottom_prod"]

kpis_df = pd.DataFrame(kpis)
kpis_df


In [ ]:

# 1) Barras: Ingresos por tienda
plt.figure(); plt.bar(kpis_df["store"], kpis_df["total_ingresos"])
plt.title("Ingresos totales por tienda"); plt.xlabel("Tienda"); plt.ylabel("Ingresos (suma de Precio)")
plt.tight_layout(); plt.savefig(FIGS/"01_ingresos_por_tienda.png", dpi=160); plt.show()

# 2) Circular: Categorías Tienda 1
cat1 = by_category.get("Tienda 1")
if cat1 is not None and len(cat1):
    plt.figure(); plt.pie(cat1["ventas"], labels=cat1.index, autopct="%1.1f%%", startangle=90)
    plt.title("Distribución de categorías — Tienda 1")
    plt.tight_layout(); plt.savefig(FIGS/"02_categorias_tienda1.png", dpi=160); plt.show()

# 3) Dispersión: Precio vs Envío
import itertools
plt.figure(); markers = itertools.cycle(["o","s","^","x"])
for name, df in stores.items():
    if df is None or df.empty: continue
    from metrics import _normalize_cols as _norm
    d = _norm(df.copy())
    if {"precio","envio"}.issubset(d.columns):
        plt.scatter(d["precio"], d["envio"], alpha=0.4, label=name, marker=next(markers))
plt.legend(title="Tienda"); plt.title("Dispersión: Precio vs Costo de envío")
plt.xlabel("Precio"); plt.ylabel("Costo de envío")
plt.tight_layout(); plt.savefig(FIGS/"03_scatter_precio_envio.png", dpi=160); plt.show()

# 4) Caja: Calificación por tienda
series_calif, labels = [], []
from metrics import _normalize_cols as _norm
for name, df in stores.items():
    if df is None or df.empty: 
        continue
    d = _norm(df.copy())
    if "calificacion" in d.columns and d["calificacion"].notna().any():
        series_calif.append(d["calificacion"].dropna())
        labels.append(name)
if series_calif:
    plt.figure(); plt.boxplot(series_calif, labels=labels)
    plt.title("Distribución de calificaciones por tienda"); plt.ylabel("Calificación")
    plt.tight_layout(); plt.savefig(FIGS/"04_boxplot_calificacion.png", dpi=160); plt.show()

# 5) Barras horizontales: Top 5 productos (Tienda 1)
tp = top_by_store.get("Tienda 1")
if tp is not None and len(tp):
    plt.figure(); tp.sort_values(ascending=True).plot(kind="barh")
    plt.title("Top 5 productos — Tienda 1 (por cantidad de ventas)"); plt.xlabel("Ventas")
    plt.tight_layout(); plt.savefig(FIGS/"05_top5_productos_tienda1.png", dpi=160); plt.show()


In [ ]:

summary = kpis_df[["store","total_ingresos","calificacion_prom","envio_promedio"]].copy()
score_df = efficiency_score(summary)
score_df

recomendada_vender = score_df.tail(1).iloc[0]["store"]
print(f"👉 Recomendación: VENDER **{recomendada_vender}** (score total más bajo).")


In [ ]:

md_lines = []
md_lines.append("# Informe final — Recomendación de tienda a vender\n")
md_lines.append("Este informe resume hallazgos y recomendación basada en datos.\n\n")
md_lines.append("## Indicadores por tienda\n\n"); md_lines.append(kpis_df.to_markdown(index=False)); md_lines.append("\n\n")
md_lines.append("## Score de eficiencia (mayor es mejor)\n\n"); md_lines.append(score_df.to_markdown(index=False)); md_lines.append("\n\n")
md_lines.append(f"## Recomendación\n\n**Vender:** {recomendada_vender}\n\n")
md_lines.append("**Motivos:** combinación de menores ingresos, peor calificación y/o mayor costo de envío relativo.\n")
md_lines.append("\n## Gráficos exportados\n")
md_lines.append("- figs/01_ingresos_por_tienda.png\n- figs/02_categorias_tienda1.png\n- figs/03_scatter_precio_envio.png\n- figs/04_boxplot_calificacion.png\n- figs/05_top5_productos_tienda1.png\n")
out_path = REPORT / "informe_final.md"
with open(out_path, "w", encoding="utf-8") as f:
    f.write("\n".join(md_lines))
print(f"Informe guardado en: {out_path}")
